## Imports

In [1]:
import geopandas as gpd
from shapely.geometry import Polygon
import numpy as np
from scipy.spatial import cKDTree

## Functions

In [2]:
def isolate_coords(shapely_coords):
    """Remove the outer 'list' from a set of shapely coordinates"""
    x = list(shapely_coords)[0][0]
    y = list(shapely_coords)[0][1]
    return [x,y]

In [4]:
def flatten_xys(arr):
    """Flatten an array into a single list for a cKDTree"""
    l = []
    for i in range(2):
        for j in range(4):
            l.append(arr[i][j])
            
    return np.array(l)


def make_ckdtree(gdf, geom_column='geometry'):
    """
    Make a cKDtree from a geodataframe.
    The cKDtree contains the exterior coordinates of your geometry.
    gdf: The geodataframe
    geom_column: The column that contains your geometry
    """
    gdf2 = gdf.copy()
    gdf2['xy'] = gdf2[geom_column].apply(lambda x:x.exterior.xy) # ony want the exterior
    n = np.array(list(gdf2['xy'].apply(flatten_xys))) # flatten the coordinates
    return cKDTree(n)

In [5]:
# https://gis.stackexchange.com/questions/222315/geopandas-find-nearest-point-in-other-dataframe

def find_orphans_new(gdf, geom_column='geometry', max_distance=1000):
    """
    Given a geodataframe with predictions, find the 'orphan' predictions.
    An orphan prediction fits one of these two conditions:
        a) There is no other prediction within max_distance kilometers
        b) There is *one* other prediction that *touches* the given prediction, and no
        other prediction is within max_distance kilometers of either.
    IMPORTANT NOTE: The EPSG **must be 3857**
    gdf: The geodataframe with predictions
    geom_column: The column with your predictions (default is geometry)
    max_distance: Default 1000 kilometers
    """
    
    tree = make_ckdtree(gdf, geom_column) # make a cKDTree to make finding the nearest prediction faster
    
    is_orphan = [True] * len(gdf) # Start with every prediction as an orphan
    index_covered = []
    
    for i in range(len(gdf)):
        print(f'Analyzing row {i+1} of {len(gdf)}', end='\r', flush=True)
        if i not in index_covered:
            poly = gdf.loc[i, geom_column]
            poly_arr = flatten_xys(poly.exterior.xy) # flatten polygon coordinates
            closest_index = tree.query(poly_arr, k=2)[1][1] # use the cKDTree to find the closest prediction
            closest_poly = gdf.loc[closest_index, geom_column] # get the polygon of the closest prediction
            distance = poly.distance(closest_poly) # find the distance between both polygons

            if distance >= max_distance:
                continue # The closest prediction is further than max_distance km away, so it's an orphan

            elif distance == 0:
                # There is another prediction that touches the current one. Need to see if there's
                # another prediction within max_distance km of either.
                poly_arr_2 = flatten_xys(closest_poly.exterior.xy)
                closest_index_1 = tree.query(poly_arr, k=3)[1][2]
                closest_index_2 = tree.query(poly_arr_2, k=3)[1][2]
                closest_poly_1 = gdf.loc[closest_index_1, geom_column]
                closest_poly_2 = gdf.loc[closest_index_2, geom_column]
                distance_1 = poly.distance(closest_poly_1)
                distance_2 = closest_poly.distance(closest_poly_2)
                # If either distance is less than max_distance, the prediction isn't an orphan.
                # Also, the polygon touching it, as well as the third polygon close to them, aren't orphans.
                # So change the requisite values in the "is_orphan" list to False, and append the other
                # two indexes to "index_covered" so the loop skips them.
                if distance_1 < max_distance:
                    is_orphan[i] = False
                    is_orphan[closest_index] = False
                    is_orphan[closest_index_1] = False
                    index_covered.append(closest_index)
                    index_covered.append(closest_index_1)
                if distance_2 < max_distance:
                    is_orphan[i] = False
                    is_orphan[closest_index] = False
                    is_orphan[closest_index_2] = False
                    index_covered.append(closest_index)
                    index_covered.append(closest_index_2)
                
            else:
                # There is another prediction within max_distance km, so neither are orphans.
                # Append the other index to index_covered so the loop skips it.
                is_orphan[i] = False
                is_orphan[closest_index] = False
                index_covered.append(closest_index)
                
    gdf2 = gdf.copy()
    gdf2['is_orphan'] = is_orphan
    return gdf2

In [ ]:
def save_nonorphans_to_file(preds_loc, new_loc, preds_crs='EPSG:4326', geom_column='geometry', max_distance=1000):
    """
    Finds the non-orphans and saves them to a new file
    preds_loc: The location of your predictions
    new_loc: Where you want to save the non-orphan predictions (including filename)
    preds_crs: The CRS of your predictions (default is EPSG:4326)
    geom_column: The column name of your prediction polygons (default is geometry)
    max_distance: If the closest prediction is more than max_distance km away, it counts as an orphan
    """

    gdf = gpd.read_file(preds_loc)
    if preds_crs != 'EPSG:3857':
        gdf = gdf.to_crs('EPSG:3857') # switch EPSG to 3857 if it isn't that already
        
    gdf_orphans = find_orphans_new(gdf, geom_column, max_distance) # find orphans
    
    pred_non_orphans = gdf_orphans[gdf_orphans['is_orphan'] == False] # extract non-orphans
    pred_non_orphans = pred_non_orphans.drop(['is_orphan'], axis=1).reset_index(drop=True)
    if preds_crs != 'EPSG:3857':
        pred_non_orphans = pred_non_orphans.to_crs(preds_crs) # switch EPSG back if necessary
        
    pred_non_orphans.to_file(new_loc, driver='GeoJSON') # save non-orphan predictions

## Example of how to use the functions

In [16]:
pred1_loc = './predictions/ISL_2019_preds.geojson'
gdf = gpd.read_file(pred1_loc)
gdf2 = gdf.to_crs('EPSG:3857') # function only works with EPSG 3857
gdf2.head()

,geometry
0,"POLYGON ((1937440.040 -286746.573, 1937440.040..."
1,"POLYGON ((1945440.015 -287747.570, 1945440.015..."
2,"POLYGON ((1942439.955 -288748.575, 1942439.955..."
3,"POLYGON ((1935439.963 -290750.605, 1935439.963..."
4,"POLYGON ((1941439.972 -292752.775, 1941439.972..."


In [17]:
gdf1_orphans = find_orphans_new(gdf2)
gdf1_orphans.head()

,geometry,is_orphan
0,"POLYGON ((1937440.040 -286746.573, 1937440.040...",True
1,"POLYGON ((1945440.015 -287747.570, 1945440.015...",True
2,"POLYGON ((1942439.955 -288748.575, 1942439.955...",True
3,"POLYGON ((1935439.963 -290750.605, 1935439.963...",True
4,"POLYGON ((1941439.972 -292752.775, 1941439.972...",True


In [18]:
gdf1_orphans['is_orphan'].value_counts()

False    21955
True     19380
Name: is_orphan, dtype: int64

In [19]:
gdf1_orphans.crs

<Derived Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World between 85.06°S and 85.06°N.
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [21]:
# Divide gdf into orphaans and non-orphans

pred1_orphans = gdf1_orphans[gdf1_orphans['is_orphan'] == True]

pred1_non_orphans = gdf1_orphans[gdf1_orphans['is_orphan'] == False]

In [22]:
assert len(gdf1_orphans) == len(pred1_orphans) + len(pred1_non_orphans)

In [24]:
pred1_non_orphans.head()

,geometry,is_orphan
10,"POLYGON ((1938440.023 -304765.962, 1938440.023...",False
21,"POLYGON ((1938440.023 -304765.962, 1938440.023...",False
24,"POLYGON ((2365839.965 474637.459, 2365839.965 ...",False
25,"POLYGON ((2366839.948 474637.459, 2366839.948 ...",False
26,"POLYGON ((2367840.042 474637.459, 2367840.042 ...",False


In [26]:
pred1_non_orphans.crs

<Derived Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World between 85.06°S and 85.06°N.
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [25]:
# Drop the now-unnecessary column and switch EPSG back

pred1_non_orphans_2 = pred1_non_orphans.drop(['is_orphan'], axis=1).reset_index(drop=True)

pred1_non_orphans_2 = pred1_non_orphans_2.to_crs('EPSG:4326')

pred1_non_orphans_2.head()

,geometry
0,"POLYGON ((17.41330 -2.73672, 17.41330 -2.72773..."
1,"POLYGON ((17.41330 -2.73672, 17.41330 -2.72773..."
2,"POLYGON ((21.25270 4.25981, 21.25270 4.26879, ..."
3,"POLYGON ((21.26168 4.25981, 21.26168 4.26879, ..."
4,"POLYGON ((21.27067 4.25981, 21.27067 4.26879, ..."


In [27]:
pred1_non_orphans_2.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [30]:
assert gdf.crs == 'EPSG:4326'

In [28]:
pred1_non_orphans_2.to_file('test.geojson', driver='GeoJSON')

In [10]:
import glob

glob.glob('./predictions/*')

['./predictions\\ISL_2019_preds.geojson',
 './predictions\\ISL_2021_preds.geojson',
 './predictions\\SAB_2019_preds.geojson',
 './predictions\\SAB_2021_preds.geojson']

In [36]:
glob.glob('./predictions/*')[0].split('\\')[-1]

'ISL_2019_preds.geojson'

In [11]:
# Loop over all predictions files

for i, preds_loc in enumerate(glob.glob('./predictions/*'), 1):
    print(f'Processing file {i} of 4')
    filename = preds_loc.split('\\')[-1].split('.')[0]
    new_filename = filename + '_nonorphans.geojson'
    new_loc = 'D:/canopy_data/analysis/DRC_preds/' + new_filename
    
    save_nonorphans_to_file(preds_loc, new_loc)
    print('A') # This is just to make the printout look pretty with the find_orphans printout

Processing file 1 of 4
Analyzing row 41335 of 41335
Processing file 2 of 4
Analyzing row 44586 of 44586
Processing file 3 of 4
Analyzing row 144615 of 144615
Processing file 4 of 4
Analyzing row 164152 of 164152
